## Time to publish!


Today we will analyze the data of the Hacker News website, and we will determine which categories are the most commented on and try to find out what is the best time for publications.

You can find the data set [https://www.kaggle.com/hacker-news/hacker-news-posts](here), but note that it has been reduced from almost 300,000 rows to approximately 20,000 rows by removing all submissions that did not receive any comments, and then randomly sampling from the remaining submissions. Below are descriptions of the columns:

| name| description|
| --- | --- |
|id: | The unique identifier from Hacker News for the post|
|title: | The title of the post|
|url: | The URL that the posts links to, if the post has a URL|
|num_points: | The number of points the post acquired|
|num_comments: | The number of comments that were made on the post|
|author: | The username of the person who submitted the post|
|created_at: | The date and time at which the post was submitted|

Let's start by importing the libraries we need and reading the data set into a list of lists. Notice that the first list in the inner lists contains the column headers, and the lists after contain the data for one row. In order to analyze our data, we need to first remove the row containing the column headers.

In [11]:
from csv import reader 
hn = list(reader(open('hacker_news.csv')))
header = hn.pop(0)


We're specifically interested in posts whose titles begin with either Ask HN or Show HN. Since we're only
concerned with post titles beginning with Ask HN or Show HN, we'll create new lists of lists containing just the data for those titles.

In [7]:
ask_posts = []
show_posts = []
other_posts = []

for row in hn:
    title = row[1]
    title = title.lower()
    if title.startswith('ask hn'):
        ask_posts.append(row)
    elif title.startswith('show hn'):
        show_posts.append(row)
    else:
        other_posts.append(row)
print(len(ask_posts))
print(len(show_posts))
print(len(other_posts))

1744
1162
17194


We separated the "ask posts" and the "show posts" into two list of lists named ask_posts and show_posts. Next, let's determine if ask posts or show posts receive more comments on average.

In [8]:
total_ask_comments = 0
for row in ask_posts:
    total_ask_comments += int(row[4])

avg_ask_comments = total_ask_comments / len(ask_posts)

total_show_comments = 0
for row in show_posts:
    total_show_comments += int(row[4])

avg_show_comments = total_show_comments / len(show_posts)

print(avg_ask_comments)
print(avg_show_comments)

14.038417431192661
10.31669535283993


Ask posts receive more comments on average, because they are created in order to get the opinion of users. So we'll focus our remaining analysis just on these posts.

Next, we'll determine if ask posts created at a certain time are more likely to attract comments. We'll use the following steps to perform this analysis:

1. Calculate the amount of ask posts created in each hour of the day, along with the number of comments received.

2. Calculate the average number of comments ask posts receive by hour created.

In [9]:
import datetime as dt
result_list = []
for row in ask_posts:
    result_list.append([row[6], int(row[4])])

counts_by_hour = {}
comments_by_hour = {}
for row in result_list:
    date = row[0]
    hour = dt.datetime.strptime(date, "%m/%d/%Y %H:%M").strftime("%H")
    comments_by_hour[hour] = comments_by_hour.get(hour, 0) + 1
    counts_by_hour[hour] = counts_by_hour.get(hour, 0) + row[1]

{'09': 251, '13': 1253, '10': 793, '14': 1416, '16': 1814, '23': 543, '12': 687, '17': 1146, '15': 4477, '21': 1745, '20': 1722, '02': 1381, '18': 1439, '03': 421, '05': 464, '19': 1188, '01': 683, '22': 479, '08': 492, '04': 337, '00': 447, '06': 397, '07': 267, '11': 641}
{'09': 45, '13': 85, '10': 59, '14': 107, '16': 108, '23': 68, '12': 73, '17': 100, '15': 116, '21': 109, '20': 80, '02': 58, '18': 109, '03': 54, '05': 46, '19': 110, '01': 60, '22': 71, '08': 48, '04': 47, '00': 55, '06': 44, '07': 34, '11': 58}


We created two dictionaries:

1. <font color='blue'>counts_by_hour</font>: contains the number of ask posts created during each hour of the day.
2. <font color='blue'>comments_by_hour</font>: contains the corresponding number of comments ask posts created at each hour received.

Next, we'll use these two dictionaries to calculate the average number of comments for posts created during each hour of the day. Let's finish by sorting the list of lists and printing the five highest values in a format that's easier to read.

In [12]:
avg_by_hour = []
for el in counts_by_hour:
    avg = counts_by_hour[el] / comments_by_hour[el]
    avg_by_hour.append([el, '%.3f' % avg])

avg_by_hour.sort()

swap_avg_by_hour =  []
for el in avg_by_hour:
    swap_avg_by_hour.append([float(el[1]), el[0]])
swap_avg_by_hour.sort(reverse=True)

print("Top 5 Hours for Ask Posts Comments")
for el in swap_avg_by_hour[:6]:
    template = '{hour}:00: {com_per_post} average comments per post'
    print(template.format(hour = el[1],com_per_post = el[0]))

Top 5 Hours for Ask Posts Comments
15:00: 38.595 average comments per post
02:00: 23.81 average comments per post
20:00: 21.525 average comments per post
16:00: 16.796 average comments per post
21:00: 16.009 average comments per post
13:00: 14.741 average comments per post
